In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *
from artificial_model import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
model_name = 'artificial'
log_name = 'train'
with open('../transformed_event_logs/artificial_start_end_2_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['1', 'Clark', 'Jane', 'Joe', 'Karsten']
known_activities = ['DIAGNOSIS', 'QUALITY_CONTROL', 'REPAIR']

In [3]:
N = 10000
n_processes = 50

import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [4]:
perfect_model = ArtificialModel()
evaluator = conduct_evaluation.ConductEvaluation(perfect_model, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_train = evaluator.sample_cases(False, True)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1802/1802 [01:18<00:00, 22.95it/s]


In [7]:
np.mean([v.ln() for v in likelihoods_train[0].values()])

Decimal('0.5026673872197590500066712016')

In [8]:
np.mean(get_pscores(likelihoods_train))

np.float64(7970.176565336295)

In [5]:
log_name = 'test'
with open('../transformed_event_logs/artificial_start_end_2_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

In [6]:
perfect_model = ArtificialModel()
evaluator = conduct_evaluation.ConductEvaluation(perfect_model, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_test = evaluator.sample_cases(False, True)

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [00:53<00:00, 16.62it/s]


In [7]:
np.mean([v.ln() for v in likelihoods_test[0].values()])

Decimal('0.5429999574310518661519231405')

In [8]:
np.mean(get_pscores(likelihoods_test))

np.float64(7968.430202444587)